In [1]:
import pandas as pd
from loguru import logger
# Setup environment for running MATLAB code if not done externally
import os
%load_ext autoreload

# os.environ["MR"] = f"{os.environ['HOME']}/PSA/MATLAB_runtime/R2023b"
os.environ["MR"] = f"{os.environ['HOME']}/MATLAB/R2023b"
MR = os.environ["MR"]
os.environ["LD_LIBRARY_PATH"] = f"{MR}/runtime/glnxa64:{MR}/bin/glnxa64:{MR}/sys/os/glnxa64:{MR}/sys/opengl/lib/glnxa64"


In [2]:
df = pd.read_csv('data/dev.csv')

In [3]:
from models_psa import SolarMED

%autoreload 2

# Initialize model
model = SolarMED(
    sample_time=60,
    curve_fits_path='data/curve_fits.json',
    
    # Initial states. Thermal storage
    Tts_h=[df['Tts_h_t'].iloc[0], df['Tts_h_m'].iloc[0], df['Tts_h_b'].iloc[0]], 
    Tts_c=[df['Tts_c_t'].iloc[0], df['Tts_c_m'].iloc[0], df['Tts_c_b'].iloc[0]],
    Tsf_in=df['Tsf_in'].iloc[0],
    
    cost_w = 3, # €/m³ 
    cost_e = 0.05 # €/kWhe
)

# Run model
for idx in range(1, 100):
    # idx = 1
    ds = df.iloc[idx]
    # df.to_csv('data/dev.csv')
        
    logger.info(f"Running iteration {idx} out of {len(df)}, last operation state was {model.operating_state}")
        
    model.step(
        # Decision variables
        ## MED
        mmed_s=ds['qmed_s'],
        mmed_f=ds['qmed_f'],
        Tmed_s_in=ds['Tmed_s_in'],
        Tmed_c_out=ds['Tmed_c_out'],
        ## Thermal storage
        mts_src=ds['qts_src'],
        ## Solar field
        Tsf_out=ds['Tsf_out'],
        
        # Inputs
        # When the solar field is starting up, a flow can be provided to sync the model with the real system, if a valid Tsf_out is provided, it will be prioritized
        msf=ds['qsf'] if ds['qsf'] > 4 else None,
        
        # Environment variables
        Tmed_c_in=ds['Tmed_c_in'],
        Tamb=ds['Tamb'],
        I=ds['I'],
    )
    
    logger.info(f'Tsf,in,mod = {model.Tsf_in:.2f}, Tsf,in,meas = {ds["Tsf_in"]:.2f}, msf_meas = {ds["qsf"]:.2f}')
    
    iteration_result = model.model_dump()

2024-02-25 20:34:15.995 | INFO     | models_psa:init_matlab_engine:784 - MATLAB engine initialized
2024-02-25 20:34:15.998 | INFO     | __main__:<module>:25 - Running iteration 1 out of 587, last operation state was SolarMED_states.IDLE
2024-02-25 20:34:15.998 | DEBUG    | models_psa.validation:within_range_or_zero_or_max:21 - (mmed_s_sp) Value 0.03 is less than 20.016 -> 0.0
2024-02-25 20:34:15.999 | DEBUG    | models_psa.validation:within_range_or_zero_or_max:21 - (mmed_f_sp) Value 0.01 is less than 5 -> 0.0
2024-02-25 20:34:16.000 | DEBUG    | models_psa.validation:within_range_or_zero_or_max:21 - (Tmed_s_in_sp) Value 17.66 is less than 60 -> 0.0
2024-02-25 20:34:16.001 | DEBUG    | models_psa.validation:within_range_or_zero_or_max:21 - (Tsf_out_sp) Value 15.31 is less than 65 -> 0.0
2024-02-25 20:34:16.002 | DEBUG    | models_psa.validation:within_range_or_zero_or_max:21 - (msf) Value 0.00 is less than 4.7 -> 0.0
2024-02-25 20:34:16.002 | DEBUG    | models_psa.validation:within_ran

TypeError: solar_field_model() got an unexpected keyword argument 'qsf'